<a href="https://colab.research.google.com/github/RoksanaMaya/MLGrit/blob/main/Amazon_Product_Review/4_FeautreEngineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
import re
from nltk import download
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
from nltk.stem import  WordNetLemmatizer

In [ ]:
download('stopwords')
download('wordnet')
download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/All_Beauty_v4.parquet')

In [ ]:
new = pd.concat([df[df.sentiment == 0].head(10000), df[df.sentiment == 1].head(10000)], ignore_index=True)
del df

In [ ]:
new.reviewLength.quantile(0.8)

54.0

# TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tf_idf = TfidfVectorizer(stop_words=stopwords.words('english'), max_features=500)
tf_idf.fit_transform(new['reviewText']).toarray().shape

(20000, 500)

### Dimesionality Reduction/ Principal Component Analysis

In [ ]:
tf_idf.fit_transform(new['reviewText'])

<20000x500 sparse matrix of type '<class 'numpy.float64'>'
	with 154762 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
reducer = PCA(n_components=50, random_state=42)

In [ ]:
reducer.fit(tf_idf.fit_transform(new['reviewText']).toarray())

PCA(n_components=50, random_state=42)

In [ ]:
reducer.transform(tf_idf.fit_transform(new['reviewText']).toarray())

array([[ 0.18736598, -0.03755091,  0.00338344, ...,  0.00087859,
        -0.06607543, -0.04153752],
       [-0.04357207, -0.03768598, -0.01637543, ...,  0.01301076,
        -0.0099984 , -0.0168232 ],
       [-0.05234466, -0.0465909 , -0.02052937, ...,  0.00530926,
        -0.0028729 ,  0.00497418],
       ...,
       [ 0.0613775 , -0.06849859, -0.01622544, ...,  0.03032442,
         0.04761762, -0.0180653 ],
       [-0.09382937, -0.01968279,  0.11877463, ...,  0.08585946,
         0.01931765,  0.06398814],
       [-0.04900108, -0.04862704, -0.01641005, ...,  0.01412451,
        -0.00554419, -0.03727783]])

In [ ]:
features = reducer.transform(tf_idf.fit_transform(new['reviewText']).toarray())
target = new.sentiment

In [ ]:
columns = [f'pc{i}'for i in range(1,51)]
df = pd.DataFrame(features, columns=columns)
df['sentiment'] = target

In [ ]:
df.head()

,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10,...,pc42,pc43,pc44,pc45,pc46,pc47,pc48,pc49,pc50,sentiment
0,0.187366,-0.037551,0.003383,-0.098317,-0.033124,0.029089,-0.003343,-0.042620,0.037142,-0.068074,...,-0.024703,0.019787,0.007909,-0.001765,-0.030302,0.066047,0.000879,-0.066075,-0.041538,0
1,-0.043572,-0.037686,-0.016375,-0.044416,-0.002662,0.003325,-0.054178,-0.034707,0.005334,-0.008146,...,-0.006265,-0.003505,-0.008676,-0.010445,0.000358,0.000371,0.013011,-0.009998,-0.016823,0
2,-0.052345,-0.046591,-0.020529,-0.052221,-0.010865,0.004682,-0.042688,-0.040378,0.008212,-0.008966,...,-0.016227,-0.010633,-0.010543,-0.018910,-0.014321,0.026418,0.005309,-0.002873,0.004974,0
3,-0.105014,0.131481,0.333232,0.031310,0.018640,-0.005778,-0.066925,0.070908,0.165601,0.037726,...,-0.016470,-0.034352,0.041027,-0.013902,0.019332,0.000592,-0.016997,-0.008004,-0.003499,0
4,-0.045729,-0.050304,-0.054038,-0.108178,0.499385,-0.025117,0.053499,0.002597,-0.013484,0.006202,...,-0.000272,0.003264,-0.004770,0.005261,0.004339,0.003538,-0.001517,-0.000754,-0.007251,0


### Save

In [ ]:
from joblib import dump

In [ ]:
dump(tf_idf, '/content/drive/MyDrive/Colab Notebooks/TFIDF')

['/content/drive/MyDrive/Colab Notebooks/TFIDF']

In [ ]:
dump(reducer, '/content/drive/MyDrive/Colab Notebooks/reducer')

['/content/drive/MyDrive/Colab Notebooks/reducer']

In [ ]:
df.to_parquet('/content/drive/MyDrive/Colab Notebooks/All_Beauty_v5.parquet')